In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()


In [ ]:
# First look in to the data size
useducation_df.shape

In [ ]:
useducation_df.info()

**Tasks**
1. Consider the two variables: *TOTAL_REVENUE* and *TOTAL_EXPENDITURE*. Do these variables have outlier values?
2. If you detect outliers in the *TOTAL_REVENUE* and *TOTAL_EXPENDITURE* variables, apply the techniques you learned in this checkpoint to eliminate them and validate that there's no outlier values after you handled them.
3. Create another variable by subtracting the original *TOTAL_EXPENDITURE* from *TOTAL_REVENUE* (before you eliminated the outliers). You can think of it as a kind of budget deficit in education. Do you find any outlier values in this new variable? If so, eliminate them using the technique you think most suitable.
4. Now create another variable by subtracting the *TOTAL_EXPENDITURE* from *TOTAL_REVENUE*. This time, use the outlier eliminated versions of *TOTAL_EXPENDITURE* from *TOTAL_REVENUE*. In this newly created variable, can you find any outliers? If so, eliminate them.
5. Compare some basic descriptive statistics of the budget variables you end up with in the 3rd and the 4th questions. Do you see any differences?
6. If our variable of interest is the budget deficit variable, which method do you think is the appropriate in dealing with the outliers in this variable: the method in the 3rd question or the one in the 4th question?

In [ ]:
# zero revenue in "STATE_REVENUE" is missing data
useducation_df=useducation_df.replace(0, np.nan)
# Replace 'VI', 'U.S._VIRGIN_ISLANDS', as 'VIRGIN_ISLAND'
useducation_df=useducation_df.replace({'VI':'VIRGIN_ISLANDS', 'U.S._VIRGIN_ISLANDS':'VIRGIN_ISLANDS', \
                                       'AS': 'AMERICAN_SAMOA', 'BI':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATIO':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATION': 'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BIE':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'DD': 'DEPARTMENT_OF_DEFENSE', \
                                       'DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY':'DEPARTMENT_OF_DEFENSE', \
                                       '^DOD.*$':'DEPARTMENT_OF_DEFENSE', \
                                       'GU':'GUAM', 'NORTHERN_MARIANAS':'NORTHERN_MARIANA_ISLANDS', \
                                       'PR':'PUERTO_RICO', 'COMMONWEALTH_OF_MARIANAS':'MARIANAS'})

useducation_df.loc[useducation_df['STATE'].str.contains('DOD'),'STATE']='DEPARTMENT_OF_DEFENSE'


In [ ]:
# show replications
display(useducation_df[useducation_df.PRIMARY_KEY.duplicated(keep='first')])

In [ ]:
# drop replications
useducation_df.drop_duplicates(subset='PRIMARY_KEY', inplace= True)

In [ ]:
column_list = ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE', \
               'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE', \
               'CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G', 'GRADES_12_G', \
               'GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', \
               'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']

useducation3_df=useducation_df.copy()
years = useducation3_df.YEAR.unique()



for column in column_list:
    for year in years:
        useducation3_df.loc[useducation3_df.YEAR==year, column].fillna( \
            useducation3_df[useducation3_df.YEAR==year][column].mean(), inplace=True)

In [ ]:
display(useducation3_df)

In [ ]:
useducation5_df=useducation3_df.copy()
useducation5_df.dropna(axis=0, subset=['TOTAL_REVENUE','TOTAL_EXPENDITURE'], inplace=True)

sns.boxplot(useducation5_df['TOTAL_REVENUE'])
plt.show()
sns.boxplot(useducation5_df['TOTAL_EXPENDITURE'])
plt.show()

In [ ]:
# Use z-score to find outliers
from scipy.stats import zscore

z_scores = zscore(useducation5_df["TOTAL_REVENUE"])
for threshold in range(1,5):
    print("The score threshold is: {}".format(threshold))
    print("The indices of the outliers:")
    print(np.where(z_scores > threshold))
    print("Number of outliers in TOTAL_REVENUE is: {}\n".format(len((np.where(z_scores > threshold)[0]))))

z_scores = zscore(useducation5_df["TOTAL_EXPENDITURE"])
for threshold in range(1,5):
    print("The score threshold is: {}".format(threshold))
    print("The indices of the outliers:")
    print(np.where(z_scores > threshold))
    print("Number of outliers is in TOTAL_EXPENDITURE: {}\n".format(len((np.where(z_scores > threshold)[0]))))


##### Yes, there are outliers on both 'TOTAL_REVENUE' and 'TOTAL_EXPENDITURE' at the high end.

In [ ]:
# Use winsorize to weed out outliers
from scipy.stats.mstats import winsorize

winsorized_REVENUE = winsorize(useducation5_df["TOTAL_REVENUE"], (0, 0.10))

sns.boxplot(winsorized_REVENUE)
plt.title('TOTAL_REVENUE')
plt.show()

winsorized_EXPENDITURE = winsorize(useducation5_df["TOTAL_EXPENDITURE"], (0, 0.10))

sns.boxplot(winsorized_EXPENDITURE)
plt.title('TOTAL_EXPENDITURE')
plt.show()

##### No more outliers after we use winsorize to process the data

In [ ]:
# New column budget_deficit = total_revenue-total_expenditure
useducation5_df['BUDGET_DEFICIT']=useducation5_df['TOTAL_REVENUE']-useducation5_df['TOTAL_EXPENDITURE']
sns.boxplot(useducation5_df['BUDGET_DEFICIT'])
plt.show()

##### The outliers in 'BUDGET_DEFICIT' are on both ends of the data.

In [ ]:
winsorized_BUDGET = winsorize(useducation5_df["BUDGET_DEFICIT"], (0.10, 0.10))

sns.boxplot(winsorized_BUDGET)
plt.title('BUDGET_DEFICIT')
plt.show()

In [ ]:
# Create Budget_decifit2 = winsorized_total_revenue-winsorized_total_expenditure
BUDGET_DEFICIT2 = winsorized_REVENUE - winsorized_EXPENDITURE
sns.boxplot(BUDGET_DEFICIT2)
plt.title('BUDGET_DEFICIT2')
plt.show()

In [ ]:
winsorized_BUDGET2 = winsorize(BUDGET_DEFICIT2, (0.10, 0.10))

sns.boxplot(winsorized_BUDGET2)
plt.title('BUDGET_DEFICIT2')
plt.show()

##### BUDGET_DEFICIT2 is created from winsorized_TOTAL_REVENUE - winsorized_TOTAL_EXPENDITURE

In [ ]:
# Basic descriptive stats

df2=pd.concat([pd.DataFrame(winsorized_BUDGET, columns=['BUDGET']), pd.DataFrame(winsorized_BUDGET2, columns=['BUDGET2'])], axis=1)
df2.describe()

The two data sets have the same counts.  We can use t-test to see if they are different.

In [ ]:
import scipy.stats as stats
stats.ttest_ind(winsorized_BUDGET, winsorized_BUDGET2)

##### The t-test shows that there is no significant difference between BUDGET and BUDGET2. (p>0.1)  

Although there is no significant difference in BUDGET and BUDGET2, it is more desirable to create the variable from original data followed by winsorization.  Therefore, BUDGET is a better choice.  